This is combined code of both the members, as we tried different descripter to explore different techniques and analyze the accuracy.

In [ ]:
# Importing libraries

import numpy as np
from keras.preprocessing import image
import cv2 as cv
import matplotlib.pyplot as plt
from pathlib import Path
import os
from skimage import feature
from sklearn.metrics import accuracy_score

From here on, Canny edge detector with flattening is used to extract the feature of the images. Later we will see the results for HOG.

In [30]:
def process(container_path): # to load images files
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]
    count = 0
    train_img = []
    counts = []
    
    for i, direc in enumerate(folders):
        s = 0
        sub_train = []
        for file in direc.iterdir():
            count += 1
            s += 1
            
            img = cv.imread(str(file),0)
            img_pred = cv.Canny(img,100,200)
            img_pred = cv.resize(img_pred, (50, 50), interpolation=cv.INTER_AREA)
            
            # Reshape the image data into rows
            img_pred = np.reshape(img_pred, (img_pred.shape[0], -1))
            img_pred = img_pred.flatten()
            
            sub_train.append(img_pred)
            
        sub_train = np.array(sub_train)
        train_img.append(sub_train)
        counts.append(s)
    
    X = np.array(train_img)
    return X,counts,categories


In [31]:
# Train dataset

X_train = []
X_train,counts,categories = process(path0+"\Training")
print("Trainig Files imported successfully")

print(len(X_train),"Categories of fruits\n")
print("Fruit--Count\n")
for i in range(len(X_train)):
    print(categories[i],"--",counts[i])

Trainig Files imported successfully
13 Categories of fruits

Fruit--Count

Apple Braeburn -- 492
Apple Crimson Snow -- 444
Avocado -- 427
Banana -- 490
Beetroot -- 450
Cactus fruit -- 490
Cantaloupe 1 -- 492
Dates -- 490
Eggplant -- 468
Grape White -- 490
Guava -- 490
Passion Fruit -- 490
Potato Red -- 450


C:\Users\USER\AppData\Local\Temp/ipykernel_21840/589802224.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(train_img)


In [32]:
fruits = []
for l in X_train:
    sm = [0]*2500
    for i in l:
        for ind,j in enumerate(i):
            sm[ind] += j
    sm = np.array(sm)
    avg = sm/2500
    fruits.append(avg)
    
fruits_mean = fruits

In [33]:
import numpy as np
from numpy import linalg as LA
import math

def cosine_distance(v1,v2): # input vectors are numpy arrays
    """"to compute cosine distance and retrn distance"""
    v1n = v1 / LA.norm(v1)
    v2n = v2 / LA.norm(v2)
    dist = 1-v1n.dot(v2n)
    return dist

def hamming(a,b): # input vectors are numpy arrays
    """"to compute hamming distance and retrn distance"""
    sum=0
    for x in range(len(a)):
        if a[x]!=b[x]:
            sum+=1
        else:
            continue
    return sum

def infnorm(a,b): # input vectors are numpy arrays
  """"to compute infinity norm distance and retrn distance"""
  d=0
  for x in range(len(a)):
    if d < abs(a[x]-b[x]):
      d=abs(a[x]-b[x])
    else:
      continue
  return d


def Pnorm(v1,v2,p=2): # input vectors are numpy arrays
  """"to compute p norm distance and retrn distance"""
  if p=="infinite":
    distance =infnorm(v1,v2)

  elif int(p)==0:
    p=int(p)
    distance =hamming(v1,v2)

  else:
    p=int(p)
    dist=0
    for i in range(np.shape(v1)[0]):
      dist = dist+(abs(v1[i]-v2[i]))**p
    distance =(dist)**(1./p)

  return distance

In [34]:
def distan(img_test):
    mi = float("inf")
    distances = []
    for ind,i in enumerate(fruits_mean):
        dist = (Pnorm(img_test,i))
        if dist <= mi :
            mi = dist
            index = ind

        distances.append(dist)
    
    return categories[index]

In [39]:
path1 = r'D:\DOC\COLLEGE DOC\NOTES\SEMESTER 4\Data analysis\fr\fruits-360\Test'

image_dir = Path(path1)
folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
categories = [fo.name for fo in folders]
count = 0
train_img = []
s = 0
y1_test = []
y1_pred = []
for i, direc in enumerate(folders):
    sub_train = []
    counts = 0
    ss = 0
    for file in direc.iterdir():
        count += 1
        counts += 1
        
        img = cv.imread(str(file),0)
        img_pred = cv.Canny(img,100,200)
            
        img_pred = cv.resize(img_pred, (50, 50), interpolation=cv.INTER_AREA)
            
        # Reshape the image data into rows
        img_pred = np.reshape(img_pred, (img_pred.shape[0], -1))
        img_pred = img_pred.flatten()
        y = distan(img_pred)
        ipred = distan(img_pred)
        y1_test.append(categories[i])
        y1_pred.append(ipred)
        if categories[i] == ipred:
            s += 1
            ss += 1
        
    print("Accuracy of",categories[i],"is",ss/counts)
print(ipred)

Accuracy of Apple Braeburn is 0.6707317073170732
Accuracy of Apple Crimson Snow is 0.05405405405405406
Accuracy of Avocado is 0.13986013986013987
Accuracy of Banana is 0.27710843373493976
Accuracy of Cactus fruit is 0.0
Accuracy of Cantaloupe 1 is 0.0
Accuracy of Dates is 0.0
Accuracy of Eggplant is 0.7692307692307693
Accuracy of Grape White is 0.0
Accuracy of Guava is 0.0
Accuracy of Passion Fruit is 0.018072289156626505
Accuracy of Potato Red is 0.19333333333333333
Eggplant


In [41]:
acc = accuracy_score(y1_test,y1_pred)
prec = precision_score(y1_test,y1_pred,average='weighted',zero_division=1)
recall = recall_score(y1_test,y1_pred,average='weighted',zero_division=1)
f1score = f1_score(y1_test,y1_pred,average='weighted')
print("Accuracy:",acc)
print("Precision:",prec)
print("Recall:",recall)
print("F1 Score:",f1score)

Accuracy: 0.17490890161374284
Precision: 0.36130015088329526
Recall: 0.17490890161374284
F1 Score: 0.13520529053365718


The expected average accuracy is very low, as consedering only edge detection; many fruits will resembel in outer shape and thus the accuracy score drops.
To improve the results we will use HOG descriptor.

In [25]:
# Path to the dataset
path0 = r"D:\DOC\COLLEGE DOC\NOTES\SEMESTER 4\Data analysis\fr\fruits-360"

This is prototype code using HOG descripter":

In [26]:
#Creating prototype for each fruit type
images = []
labels = []
prot = []
image_paths = os.listdir(f"D:/DOC/COLLEGE DOC/NOTES/SEMESTER 4/Data analysis/fr/fruits-360/Training")
for path in image_paths:
    all_images = os.listdir(f"D:/DOC/COLLEGE DOC/NOTES/SEMESTER 4/Data analysis/fr/fruits-360/Training/{path}")
    images = []
    means = []
    for image in all_images:
        image_path = f"D:/DOC/COLLEGE DOC/NOTES/SEMESTER 4/Data analysis/fr/fruits-360/Training/{path}/{image}"
        image = cv.imread(image_path)
        image = cv.resize(image, (64,64),interpolation=cv.INTER_AREA)
        image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        hog_desc = feature.hog(image, orientations=9, pixels_per_cell=(8, 8),
            cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')
        if len(means)==0:
            means=hog_desc
        else:
            for i in range(len(means)):
                means[i]=means[i]+hog_desc[i]
    cnt=len(means)
    for i in range(len(means)):
        means[i]=means[i]/cnt
    prot.append(means)
    labels.append(path)
    print(path)


Apple Braeburn
Apple Crimson Snow
Avocado
Banana
Beetroot
Cactus fruit
Cantaloupe 1
Dates
Eggplant
Grape White
Guava
Passion Fruit
Potato Red


Predicting the result using the prtotype method:

In [42]:
test_image_paths = os.listdir(f"D:/DOC/COLLEGE DOC/NOTES/SEMESTER 4/Data analysis/fr/fruits-360/Test")
prediction=[]
test_label= []
y2_test= []
y2_pred = []
for path in test_image_paths:
    # get all the image names
    all_test_images = os.listdir(f"D:/DOC/COLLEGE DOC/NOTES/SEMESTER 4/Data analysis/fr/fruits-360/Test/{path}")
    # iterate over the image names, get the label
    image_pred=[]
    for image in all_test_images:
        image_path = f"D:/DOC/COLLEGE DOC/NOTES/SEMESTER 4/Data analysis/fr/fruits-360/Test/{path}/{image}"
        image = cv.imread(image_path)
        image = cv.resize(image, (64, 64))
        image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        # get the HOG descriptor for the image
        hog_desc = feature.hog(image, orientations=9, pixels_per_cell=(8, 8),
            cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')
        # update the data and labels
        dist=[]
        for i in prot:
            dist.append(np.linalg.norm(i-hog_desc))
        ipred = labels[dist.index(min(dist))]
        image_pred.append(ipred)
        y2_test.append(path)
        y2_pred.append(ipred)
    prediction.append(image_pred)
    print(path)
    test_label.append(path)
    

Apple Braeburn
Apple Crimson Snow
Avocado
Banana
Cactus fruit
Cantaloupe 1
Dates
Eggplant
Grape White
Guava
Passion Fruit
Potato Red


In [43]:
def accuracy(pred,real):
    if len(real)==1:
        real=real*len(pred)
    return accuracy_score(pred,real)
s=0
for i in range(len(test_label)):
    acc = accuracy(prediction[i],[test_label[i]])
    s = s + acc
    print(f"Accuracy for the prediction of class {test_label[i]} is ",acc)
print("\nAverage is ",s/len(test_label))

Accuracy for the prediction of class Apple Braeburn is  0.5853658536585366
Accuracy for the prediction of class Apple Crimson Snow is  0.10810810810810811
Accuracy for the prediction of class Avocado is  0.42657342657342656
Accuracy for the prediction of class Banana is  0.608433734939759
Accuracy for the prediction of class Cactus fruit is  0.5
Accuracy for the prediction of class Cantaloupe 1 is  1.0
Accuracy for the prediction of class Dates is  0.5
Accuracy for the prediction of class Eggplant is  0.0
Accuracy for the prediction of class Grape White is  0.5
Accuracy for the prediction of class Guava is  0.5662650602409639
Accuracy for the prediction of class Passion Fruit is  0.6626506024096386
Accuracy for the prediction of class Potato Red is  0.0

Average is  0.4547830654942027


In [44]:
acc = accuracy_score(y2_test,y2_pred)
prec = precision_score(y2_test,y2_pred,average='weighted',zero_division=1)
recall = recall_score(y2_test,y2_pred,average='weighted',zero_division=1)
f1score = f1_score(y2_test,y2_pred,average='weighted')
print("Accuracy:",acc)
print("Precision:",prec)
print("Recall:",recall)
print("F1 Score:",f1score)

Accuracy: 0.46382092660072877
Precision: 0.6690228851400865
Recall: 0.46382092660072877
F1 Score: 0.4577906449219629


The average accuracy improves drastically in case of the HOG desscriptor, consedering it can be due to the fact that HOG uses both angle and magnitude of the gradient.